# MobileNet + 计算量问题

## 技巧+实现
- depthwise conv 深度可分离卷积
    - 分组卷积 + 分到极致 + kernel规模相同
    - 首先通过一个输出通道为1的1\*1 非线性变换 使得每个卷积输入通道为1
    - 减少了参数 减少了计算量
    - 目标：在移动设备等上 减少计算量
    - 问题：精度有所损失 <10%
- 等价模型：
    - 3\*3 + bn + relu == depthwise 3\*3 conv + bn + relu + 1\*1 + bn + relu
    
- 计算量问题：
    - 参数!=计算量(ci co输入输出通道数目)
    - 卷积参数 ci\*co\*kw\*kh
    - 计算量 ci\*co\*kw\*kh\*ow\*oh
    - **mobileNet优化**：
        - 深度可分离层：kw\*kh\*ow\*oh\*1\*ci (ci表示多个1输出拼接 一共有ci个 本身的输出通道1)
        - 1\*1卷积 1\*1\*ow\*oh\*ci\*co (ci为合成好的 co为输出的通道数目)
        - 将以上计算量相加为总计算量
        
- 实现技巧
    - 使用split进行拆分
    - 使用concat进行组合
    - 使用scope加以区分
    - 1\*1 后置
    
> 当成普通卷积层就可以了 主要目标是加速运算 
只需要将原来的3\*3 通过分组合并+后置1\*1实现变换


In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


### mobilenet

In [4]:
def separable_block(x,output_channel_num,name):
    """inception block implementation"""
    """
    Args:
    - output_channel_num: output_channel of 1*1 con layer
    - name: 区分当前inception block
    - 此处使用1*1 3*3 5*5 + maxpooling
    """
    with tf.variable_scope(name):
        input_channel = x.get_shape().as_list()[-1]
        # 分拆后的x
        channel_wise_x = tf.split(x,input_channel,axis=3)
        outputs = []
        
        for i in range(len(channel_wise_x)):
            output = tf.layers.conv2d(channel_wise_x[i],
                                        1,
                                        (3,3),
                                        strides=(1,1),padding='same',
                                        activation=tf.nn.relu,
                                        name='conv%d'%i)
            outputs.append(output)
            
        # concat
        concat_layer = tf.concat(outputs,axis=3)
        # conv1_1 处理
        conv1_1 = tf.layers.conv2d(concat_layer,output_channel_num,
                           (1,1),strides=(1,1),padding='same',
                           activation=tf.nn.relu,name='conv1_1')
        
        return conv1_1

### model

In [5]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
# model constitute
conv1 = tf.layers.conv2d(x_image,32,(3,3),padding='same',
                         activation=tf.nn.relu,name='conv1')
pooling1 = tf.layers.max_pooling2d(conv1,(2,2),(2,2),name='pool1')


separable_2a = separable_block(pooling1,64,name='separable_2a')
separable_2b = separable_block(separable_2a,64,name='separable_2b')
pooling2 = tf.layers.max_pooling2d(separable_2b,(2,2),(2,2),name='pool2')


separable_3a = separable_block(pooling2,128,name='separable_3a')
separable_3b = separable_block(separable_3a,128,name='separable_3b')
pooling3 = tf.layers.max_pooling2d(separable_3b,(2,2),(2,2),name='pool3')

# 保持batchsize维度 其他维度展平 便于fc
flatten = tf.layers.flatten(pooling3)
print(flatten.get_shape(),pooling3.get_shape())

y_predict = tf.layers.dense(flatten,10,name='fc_output')

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_predict)

correct_prediction = tf.equal(tf.argmax(y_predict,1),y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.variable_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

(?, 2048) (?, 4, 4, 128)


### load data

In [7]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        # hstack vstack 用于合并二维数组变成矩阵形式
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        # 小心
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


### train

In [8]:
# cfg = tf.ConfigProto(allow_soft_placement=True )
# cfg.gpu_options.allow_growth = True
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 

init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100

# train 10k: 71.35%
with tf.Session(config=config) as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 50 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 200 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 50, loss: 2.10032, acc: 0.25000
[Train] Step: 100, loss: 2.25024, acc: 0.10000
[Train] Step: 150, loss: 1.97786, acc: 0.25000
[Train] Step: 200, loss: 2.23641, acc: 0.15000
(10000, 3072)
(10000,)
[Test ] Step: 200, acc: 0.27500
[Train] Step: 250, loss: 1.87660, acc: 0.20000
[Train] Step: 300, loss: 1.56968, acc: 0.45000
[Train] Step: 350, loss: 1.44084, acc: 0.40000
[Train] Step: 400, loss: 1.55556, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step: 400, acc: 0.34850
[Train] Step: 450, loss: 1.82178, acc: 0.40000
[Train] Step: 500, loss: 1.47369, acc: 0.55000
[Train] Step: 550, loss: 1.47868, acc: 0.40000
[Train] Step: 600, loss: 1.40704, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] Step: 600, acc: 0.43900
[Train] Step: 650, loss: 1.86479, acc: 0.40000
[Train] Step: 700, loss: 1.35214, acc: 0.40000
[Train] Step: 750, loss: 2.24117, acc: 0.20000
[Train] Step: 800, loss: 1.57967, acc: 0.30000
(10000, 3072)
(10000,)
[Test ] Step: 800, acc: 0.46850
[Train] Step: 850, loss: 1.31